In [ ]:
import mne
import os
import gc
import psutil

import utils.config as config
from utils.helpers import perform_ica_cleaning, format_numbers
from utils.file_io import save_epochs, load_epochs
from utils.config import DATASETS

%matplotlib qt

### Defining constants and preparing stuff ###

DATASET = DATASETS["braboszcz2017"]

subfolder_input = "preprocessed"
subfolder_output = "ica_cleaned"
path_epochs = DATASET.path_epochs
path_epochs_preprocessed = os.path.join(path_epochs, subfolder_input)

# EEG settings
subjects = format_numbers(DATASET.subjects, 3)
sessions = DATASET.sessions

process = psutil.Process()

In [ ]:
# Load in the epochs and do ICA
for subject in subjects:
    for session in sessions:
        epochs = load_epochs(path_epochs_preprocessed, subject, session)

        # Skip if epochs are not loaded
        if epochs is None:
            continue

        # Extract epochs for each session
        epochs_med2 = epochs["med2"]
        epochs_think2 = epochs["think2"]

        # Perform ICA on med2 epochs
        cleaned_epochs_med2, ica_med2 = perform_ica_cleaning(epochs_med2, subject)

        # Perform ICA on think2 epochs
        cleaned_epochs_think2, ica_think2 = perform_ica_cleaning(epochs_think2, subject)

        # concatenate the epochs
        epochs_concat = mne.concatenate_epochs([cleaned_epochs_med2, cleaned_epochs_think2])

        # Save the epochs
        save_epochs(epochs_concat, path_epochs, subject, subfolder=subfolder_output)
        
        del epochs
        gc.collect()
        print(f"Memory after cleanup: {process.memory_info().rss / 1e6:.2f} MB")